In [1]:
import tensorflow as tf
from tensorflow import keras
from component.respiratory_tract import get_respiratory_tract_model
from utils.feature_extraction import  get_MFCCS_v2
from sklearn.utils import class_weight
import numpy as np

In [2]:
n_mfccs = 39
num_segments = 70

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size, n_mfccs, num_segments):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = set(np.arange(len(X)))
        self.num_segments = num_segments
        self.n_mfccs = n_mfccs

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        X = []
        y = []
        for index in indexes:
            try:
                X.append(get_MFCCS_v2(self.X[index], n_mfccs=self.n_mfccs, num_segments=self.num_segments))
                y.append(self.y[index])
            except:
                continue
        return np.array(X), np.array(y, dtype=int)


In [4]:
import glob
X1 = glob.glob("coswara_wavs/train/p/*.wav")
X0 = glob.glob("coswara_wavs/train/n/*.wav")
X = X1 + X0
y = [1]*len(X1) + [0] * len(X0)
print(f"Found {len(X1)} label positive")
print(f"Found {len(X0)} label negative")

Found 296 label positive
Found 438 label negative


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)
class_weights = {1 - i: x for i,x in enumerate(class_weights)}
print(f"Class weights: {class_weights}")

Class weights: {1: 0.8378995433789954, 0: 1.239864864864865}


d:\project\papyrus\venv\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1], y=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [6]:
train_generator = DataGenerator(X_train, y_train, 8, n_mfccs, num_segments)
val_generator = DataGenerator(X_val, y_val, 8, n_mfccs, num_segments)

In [7]:
model = get_respiratory_tract_model(input_shape=(n_mfccs*3+3,num_segments,1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])

In [8]:
history = model.fit(train_generator, steps_per_epoch=20, validation_steps=20, epochs=200, validation_data=val_generator, class_weight=class_weights,
                    callbacks=[tf.keras.callbacks.ModelCheckpoint('pretrained_weight/model_new_paper.h5', monitor="val_acc", save_weights_only=True, verbose=1, mode='max', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir='logs')])

Epoch 1/200
20/20 [==============================] - 18s 397ms/step - loss: 1.6324 - acc: 0.4055 - val_loss: 0.7566 - val_acc: 0.3949

Epoch 00001: val_acc improved from -inf to 0.39490, s

In [9]:
from sklearn.metrics import classification_report
import tqdm
X1_test = glob.glob("coswara_wavs/test/p/*.wav")
X0_test = glob.glob("coswara_wavs/test/n/*.wav")
X_test = []
y_test = []
for audio_path in tqdm.tqdm(X1_test):
    try:
        X_test.append(get_MFCCS_v2(audio_path, n_mfccs=n_mfccs, num_segments=num_segments))
        y_test.append(1)
    except:
        continue

for audio_path in tqdm.tqdm(X0_test):
    try:
        X_test.append(get_MFCCS_v2(audio_path, n_mfccs=n_mfccs, num_segments=num_segments))
        y_test.append(0)
    except:
        continue
X_test = np.array(X_test)

100%|██████████| 514/514 [00:00<00:00, 609.50it/s]


In [10]:
model.load_weights("pretrained_weight/model_new_paper.h5")
y_pred = model.predict(X_test)
y_pred_r = np.rint(y_pred)
print(classification_report(y_pred_r, y_test))

              precision    recall  f1-score   support

         0.0       0.71      0.90      0.80       400
         1.0       0.47      0.20      0.28       179

    accuracy                           0.68       579
   macro avg       0.59      0.55      0.54       579
weighted avg       0.64      0.68      0.64       579



In [15]:
y_pred

array([[0.28859642],
       [0.27104372],
       [0.27789068],
       [0.26825005],
       [0.26863956],
       [0.27381045],
       [0.23402883],
       [0.25647363],
       [0.25250924],
       [0.24199101],
       [0.25727716],
       [0.25386164],
       [0.24968651],
       [0.26107177],
       [0.24173716],
       [0.23896241],
       [0.26831368],
       [0.26226237],
       [0.23609175],
       [0.25913936],
       [0.24503745],
       [0.22692668],
       [0.27399725],
       [0.2662678 ],
       [0.29808724],
       [0.23121552],
       [0.27892596],
       [0.27026004],
       [0.2618421 ],
       [0.25730377],
       [0.23501246],
       [0.2393519 ],
       [0.26955828],
       [0.24567617],
       [0.2395777 ],
       [0.23218304],
       [0.2507535 ],
       [0.27468354],
       [0.25604925],
       [0.23694496],
       [0.27407682],
       [0.2625519 ],
       [0.28280193],
       [0.28332376],
       [0.24754553],
       [0.2735111 ],
       [0.2650875 ],
       [0.241